<a href="https://colab.research.google.com/github/PereiraVagner/Challenges/blob/main/Analise_Eleicoes_RenovaBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# instalação de pacotes
!pip install gcsfs
!pip install pandera
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [15]:
# importação de módulos
import os                              # Importar a biblioteca "os" para interagir com o sistema operacional
from google.colab import drive         # Esse modulo fornece funcionalidades para interagir com a ferramenta escrever scritps do Google na nuvem
from  google.cloud import storage      # Esse módulo fornece funcionalidades para interagir com o serviço de armazenamento em nuvem do Google Cloud Platform (GCP)


In [16]:
# Ignorar avisos | ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Importando as ferramentas e criando o ambiente virtual para o spark

In [17]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Para deixar a visualição das tabelas mais amigável
#from pyspark.sql.functions import sum, when
from pyspark.sql.functions import sum, max, when, col, row_number
from pyspark.sql.functions import count, isnan, isnull, sum, isnan
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType # importando estrutura da tabela (structType)
from pyspark.sql.window import Window
from pyspark.sql import functions as F
spark

# Extração

* Google Drive


In [19]:
#Comando para conexão/abertura Google Drive | command to coonect/open Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
Não foi utilizando devido ao tamanho dos arquivos

#PANDA
# Extração via Google Drive - via Google cloud usando variável PATH | variable to extract the Drive Path or the GCloud Path

# DB Perfil Eleitor
path_perfil_eleitor = '/content/drive/MyDrive/Jobs/Linkedln/database_desfio_renovaBR/eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv'
df_perfil = pd.read_csv(path_perfil_eleitor,
                      sep=';',
                      encoding='ISO-8859-1',
                      dayfirst = True)

# DB Resultado Eleições
path_resultado_eleicoes = '/content/drive/MyDrive/Jobs/Linkedln/database_desfio_renovaBR/resultados/SP_turno_1.csv'
df_resultado = pd.read_csv(path_resultado_eleicoes,
                      sep=';',
                      encoding='ISO-8859-1',
                      dayfirst = True)
'''

In [20]:
# Extração utlizando pySpark

# Realizando leitura dos dados: resultado eleições
df_resultado = spark.read.format("csv")\
    .option("delimiter", ";")\
    .option("encoding", "ISO-8859-1")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load('/content/drive/MyDrive/Jobs/Linkedln/database_desfio_renovaBR/resultados/SP_turno_1.csv')

# Realizando leitura dos dados: perfil eleitor
df_eleitorado = spark.read.format("csv")\
    .option("delimiter", ";")\
    .option("encoding", "ISO-8859-1")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load('/content/drive/MyDrive/Jobs/Linkedln/database_desfio_renovaBR/eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv')

# Pré-Análise

In [23]:
#Criação do Backup local do Dataframe | Local Backup Dataframe
df_eleitorado_bkp = df_eleitorado.select("*")

df_resultado_bkp = df_resultado.select("*")

In [21]:
# Mostrando os dados
df_resultado.show(5)

+----------+----------+-----------+---------------+-----------------+---------+----------+--------+----------+--------------------+-----+------------+------------+-------+--------+----------------+-----------------+-----------------+----------+----------+--------------------+-------------------+--------+-----------------+-------------+------------+------------+---------------+---------------+----------+-------------+--------+-----------------+-------------------------+-------------------------+---------------------------+-----------------------+-----------------------+------------+-------------------+-------------------+-------------------------+-------------------+------------------+------------------+
|DT_GERACAO|HH_GERACAO|ANO_ELEICAO|CD_TIPO_ELEICAO|  NM_TIPO_ELEICAO|CD_PLEITO| DT_PLEITO|NR_TURNO|CD_ELEICAO|          DS_ELEICAO|SG_UF|CD_MUNICIPIO|NM_MUNICIPIO|NR_ZONA|NR_SECAO|NR_LOCAL_VOTACAO|CD_CARGO_PERGUNTA|DS_CARGO_PERGUNTA|NR_PARTIDO|SG_PARTIDO|          NM_PARTIDO|     DT_BU_

In [22]:
# Mostrando os dados
df_eleitorado.show(5)

+----------+----------+-----------+-----+------------+--------------------+--------------------+--------------------+-------+---------+---------+---------------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------------------+--------------------------+
|DT_GERACAO|HH_GERACAO|ANO_ELEICAO|SG_UF|CD_MUNICIPIO|        NM_MUNICIPIO|CD_MUN_SIT_BIOMETRIA|DS_MUN_SIT_BIOMETRIA|NR_ZONA|CD_GENERO|DS_GENERO|CD_ESTADO_CIVIL|DS_ESTADO_CIVIL|CD_FAIXA_ETARIA|     DS_FAIXA_ETARIA|CD_GRAU_ESCOLARIDADE|DS_GRAU_ESCOLARIDADE|QT_ELEITORES_PERFIL|QT_ELEITORES_BIOMETRIA|QT_ELEITORES_DEFICIENCIA|QT_ELEITORES_INC_NM_SOCIAL|
+----------+----------+-----------+-----+------------+--------------------+--------------------+--------------------+-------+---------+---------+---------------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------

In [ ]:
#df_eleitorado.info(verbose=True)
df_eleitorado.describe()

summary,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
count,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507,4248507
mean,null,null,2020.0,null,51853.41377759293,null,0.0,null,99.2367723532055,3.039348646477457,null,3.8778707437695172,null,5279.178914851735,null,4.243421983299074,null,34.8165798008571,27.67912939769194,0.2726214173590864,0.002350237389275...
stddev,null,null,0.0,null,28381.17776710989,null,0.0,null,91.3504192051455,1.0199620766587194,null,2.8375157773663684,null,2105.560617107195,null,2.2195559671343035,null,121.13255713874497,99.75691315261516,1.6958752269938204,0.053198321493675704
min,01/08/2020,08:37:48,2020,AC,19,ABADIA DE GOIÁS,0,Sem biometria,1,0,FEMININO,0,CASADO,-3,100 anos ou mais ...,0,ANALFABETO,1,0,0,0
max,01/08/2020,08:37:48,2020,TO,99074,ÓLEO,0,Sem biometria,427,4,NÃO INFORMADO,9,VIÚVO,9999,Inválido ...,8,SUPERIOR INCOMPLETO,6438,6437,383,7


# Seleção das Colunas que serão utilizadas

In [ ]:
# Selecionando as colunas desejadas, realizando um DROP das que Não iremos usar
df_result_final = df_resultado.drop("DT_GERACAO","HH_GERACAO","CD_TIPO_ELEICAO","DT_PLEITO","CD_ELEICAO","CD_CARGO_PERGUNTA","DT_BU_RECEBIDO","NR_VOTAVEL",
                                    "QT_APTOS|","QT_COMPARECIMENTO","QT_ABSTENCOES","DS_TIPO_URNA","CD_TIPO_URNA","CD_CARGA_1_URNA_EFETIVADA","CD_CARGA_2_URNA_EFETIVADA",
                                    "CD_FLASHCARD_URNA_EFETIVADA","DT_CARGA_URNA_EFETIVADA","DS_CARGO_PERGUNTA_SECAO","DS_AGREGADAS","DT_ABERTURA",
                                    "DT_ENCERRAMENTO","DT_EMISSAO_BU","NR_JUNTA_APURADORA","NR_TURMA_APURADORA","CD_MUNICIPIO","CD_PLEITO", "NR_SECAO",
                                    "NM_TIPO_ELEICAO","NR_LOCAL_VOTACAO","NR_PARTIDO","NM_PARTIDO","QT_APTOS","CD_TIPO_VOTAVEL","NR_URNA_EFETIVADA")

In [ ]:
# Verificando se o Schema está correto
df_result_final.printSchema()

root
 |-- ANO_ELEICAO: integer (nullable = true)
 |-- NR_TURNO: integer (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- NM_MUNICIPIO: string (nullable = true)
 |-- NR_ZONA: integer (nullable = true)
 |-- DS_CARGO_PERGUNTA: string (nullable = true)
 |-- SG_PARTIDO: string (nullable = true)
 |-- DS_TIPO_VOTAVEL: string (nullable = true)
 |-- NM_VOTAVEL: string (nullable = true)
 |-- QT_VOTOS: integer (nullable = true)
 |-- QT_ELEITORES_BIOMETRIA_NH: integer (nullable = true)



In [ ]:
# Renomeia as antigas do df_result_final, armazena no mesmo df - Overwrite
'''
Caso queira renomear as colunas, eu decidi trabalhar com os dados brutos

df_result_final = df_result_final\
    .withColumnRenamed("ANO_ELEICAO", "ano")\
    .withColumnRenamed("NR_TURNO", "turno")\
    .withColumnRenamed("SG_UF", "estado")\
    .withColumnRenamed("NM_MUNICIPIO", "municipio")\
    .withColumnRenamed("NR_ZONA", "zona")\
    .withColumnRenamed("DS_CARGO_PERGUNTA", "cargo")\
    .withColumnRenamed("SG_PARTIDO", "partido")\
    .withColumnRenamed("DS_TIPO_VOTAVEL", "tipo_voto")\
    .withColumnRenamed("NM_VOTAVEL", "candidato")\
    .withColumnRenamed("QT_VOTOS", "qtd_voto")\
    .withColumnRenamed("QT_ELEITORES_BIOMETRIA_NH", "qt_eleitor_biometria")
'''

'\nCaso queira renomear as colunas, eu decidi trabalhar com os dados brutos\n\ndf_result_final = df_orddf_result_finaler_payments    .withColumnRenamed("ANO_ELEICAO", "ano")    .withColumnRenamed("NR_TURNO", "turno")    .withColumnRenamed("SG_UF", "estado")    .withColumnRenamed("NM_MUNICIPIO", "municipio")    .withColumnRenamed("NR_ZONA", "zona")    .withColumnRenamed("DS_CARGO_PERGUNTA", "cargo")    .withColumnRenamed("SG_PARTIDO", "partido")    .withColumnRenamed("DS_TIPO_VOTAVEL", "tipo_voto")    .withColumnRenamed("NM_VOTAVEL", "candidato")    .withColumnRenamed("QT_VOTOS", "qtd_voto")    .withColumnRenamed("QT_ELEITORES_BIOMETRIA_NH", "qt_eleitor_biometria")\n'

In [ ]:
'''
Por algum motivo, não esta filtrando os dados de SP
# Selecionado somente dados de SP
df_result_final = df_result_final.select("*").filter(
    (df_result_final.SG_UF == "SP")
)
'''

In [ ]:
# Selecionando as colunas desejadas, realizando um DROP das que Não iremos usar
df_eleitorado_final = df_eleitorado.drop(
    "DT_GERACAO","HH_GERACAO","ANO_ELEICAO","CD_MUNICIPIO","CD_MUN_SIT_BIOMETRIA","CD_GENERO",
    "CD_ESTADO_CIVIL","CD_FAIXA_ETARIA","CD_GRAU_ESCOLARIDADE","QT_ELEITORES_INC_NM_SOCIAL"
)

In [ ]:
# Verificando se o Schema está correto
df_eleitorado_final.printSchema()

root
 |-- SG_UF: string (nullable = true)
 |-- NM_MUNICIPIO: string (nullable = true)
 |-- DS_MUN_SIT_BIOMETRIA: string (nullable = true)
 |-- NR_ZONA: integer (nullable = true)
 |-- DS_GENERO: string (nullable = true)
 |-- DS_ESTADO_CIVIL: string (nullable = true)
 |-- DS_FAIXA_ETARIA: string (nullable = true)
 |-- DS_GRAU_ESCOLARIDADE: string (nullable = true)
 |-- QT_ELEITORES_PERFIL: integer (nullable = true)
 |-- QT_ELEITORES_BIOMETRIA: integer (nullable = true)
 |-- QT_ELEITORES_DEFICIENCIA: integer (nullable = true)



In [ ]:
# Renomeia as antigas do df_eleitorado_final, armazena no mesmo df - Overwrite
'''
Caso queira renomear as colunas, eu decidi trabalhar com os dados brutos

df_eleitorado_final = df_eleitorado_final\
    .withColumnRenamed("SG_UF", "estado")\
    .withColumnRenamed("DS_MUN_SIT_BIOMETRIA", "biometria_municipio")\
    .withColumnRenamed("NR_ZONA", "zona")\
    .withColumnRenamed("DS_GENERO", "genero")\
    .withColumnRenamed("DS_ESTADO_CIVIL", "estado_civil")\
    .withColumnRenamed("DS_FAIXA_ETARIA", "faixa_etaria")\
    .withColumnRenamed("DS_GRAU_ESCOLARIDADE", "escolaridade")\
    .withColumnRenamed("QT_ELEITORES_PERFIL", "qt_perfil_eleitor")\
    .withColumnRenamed("QT_ELEITORES_BIOMETRIA", "qt_eleitor_biometria")
    .withColumnRenamed("QT_ELEITORES_DEFICIENCIA", "qt_eleitor_deficiencia")
'''

In [ ]:
# Salvar p  DataFrame em um arquivo CSV
#output_path = "/Users/vg-p/Documents/Vg_Personal/RenovaBR/database_desfio_renovaBR/resultados/eleitorado_test.csv"
#df_eleitorado_final.write.option("header", True).csv(output_path)

# Transformação

**Verificações gerais e básicas de qualquer base de dados, se houver os casos abaixo:**

*  Escolher as possíveis colunas importantes para a análise;
*  Verificar se há colunas colunas iguais;
*  Verificação e tratamento de valores (ou observações) únicos;
*  Verificação e tratamento de duplicadas
*  Verificação e tatamento de inconsistência
*  Verificação e tratamento de valores nulos
*  Tradução/Renomeação
*  Transformação de tipos adequados dos dados
*  Garantia de qualidade e integridade dos dados

In [ ]:
# Verificar valores nulos | Check for null values in each column
print("Contagem de valores nulos por coluna:")
df_result_final.agg(*[(count(when(isnull(col(c)), c)).alias(c)) for c in df_result_final.columns]).show()
df_eleitorado_final.agg(*[(count(when(isnull(col(c)), c)).alias(c)) for c in df_eleitorado_final.columns]).show()


# Contagem de valores ausentes por coluna | Check empty values in each column
print("Contagem de valores ausentes por coluna:")
df_result_final.agg(*[(count(when(isnan(col(c)) | isnull(col(c)), c)).alias(c)) for c in df_result_final.columns]).show()
df_eleitorado_final.agg(*[(count(when(isnan(col(c)) | isnull(col(c)), c)).alias(c)) for c in df_eleitorado_final.columns]).show()

# Contagem de valores não nulos por coluna
#df_result_final.agg(*[(count(when(col(c).isNotNull(), c)).alias(c)) for c in df_result_final.columns]).show()
#null_columns = [col for col in df_result_final.columns if df_result_final.filter(F.col(col).isNull()).count() > 0]

Contagem de valores nulos por coluna:
+-----------+--------+----------+-----+------------+-------+-----------------+----------+---------------+----------+--------+-------------------------+
|ANO_ELEICAO|NR_TURNO|DS_ELEICAO|SG_UF|NM_MUNICIPIO|NR_ZONA|DS_CARGO_PERGUNTA|SG_PARTIDO|DS_TIPO_VOTAVEL|NM_VOTAVEL|QT_VOTOS|QT_ELEITORES_BIOMETRIA_NH|
+-----------+--------+----------+-----+------------+-------+-----------------+----------+---------------+----------+--------+-------------------------+
|          0|       0|         0|    0|           0|      0|                0|         0|              0|         0|       0|                        0|
+-----------+--------+----------+-----+------------+-------+-----------------+----------+---------------+----------+--------+-------------------------+

+-----+------------+--------------------+-------+---------+---------------+---------------+--------------------+-------------------+----------------------+------------------------+
|SG_UF|NM_MUNICIPIO|

In [24]:
# Verificar Valores Discrepantes das colunas - separado por pares
df_eleitorado_valores =(
    df_eleitorado.select('NM_MUNICIPIO', 'DS_GENERO')
    .distinct()
    .orderBy(F.col('NM_MUNICIPIO'), F.col('DS_GENERO'))
    .rdd.flatMap(lambda x: x)
    .collect()
)

# Mostrar os valores distintos
print(sorted(map(str, df_eleitorado_valores)))

['ABADIA DE GOIÁS', 'ABADIA DE GOIÁS', 'ABADIA DOS DOURADOS', 'ABADIA DOS DOURADOS', 'ABADIA DOS DOURADOS', 'ABADIÂNIA', 'ABADIÂNIA', 'ABAETETUBA', 'ABAETETUBA', 'ABAETETUBA', 'ABAETÉ', 'ABAETÉ', 'ABAETÉ', 'ABAIARA', 'ABAIARA', 'ABARÉ', 'ABARÉ', 'ABARÉ', 'ABATIÁ', 'ABATIÁ', 'ABAÍRA', 'ABAÍRA', 'ABDON BATISTA', 'ABDON BATISTA', 'ABEL FIGUEIREDO', 'ABEL FIGUEIREDO', 'ABELARDO LUZ', 'ABELARDO LUZ', 'ABRE CAMPO', 'ABRE CAMPO', 'ABREU E LIMA', 'ABREU E LIMA', 'ABREU E LIMA', 'ABREULÂNDIA', 'ABREULÂNDIA', 'ACAIACA', 'ACAIACA', 'ACAIACA', 'ACAJUTIBA', 'ACAJUTIBA', 'ACARAPE', 'ACARAPE', 'ACARAÚ', 'ACARAÚ', 'ACARI', 'ACARI', 'ACARÁ', 'ACARÁ', 'ACAUÃ', 'ACAUÃ', 'ACEGUÁ', 'ACEGUÁ', 'ACOPIARA', 'ACOPIARA', 'ACOPIARA', 'ACORIZAL', 'ACORIZAL', 'ACRELÂNDIA', 'ACRELÂNDIA', 'ACREÚNA', 'ACREÚNA', 'ADAMANTINA', 'ADAMANTINA', 'ADAMANTINA', 'ADELÂNDIA', 'ADELÂNDIA', 'ADOLFO', 'ADOLFO', 'ADRIANÓPOLIS', 'ADRIANÓPOLIS', 'ADUSTINA', 'ADUSTINA', 'AFOGADOS DA INGAZEIRA', 'AFOGADOS DA INGAZEIRA', 'AFONSO BEZERRA'

In [13]:
'''
Para realizar de todas as colunas é necessário poder computacional

# Verificar Valores Discrepantes das colunas
df_eleitorado_valores =(
    df_eleitorado.select('NM_MUNICIPIO', 'DS_GENERO', 'SG_UF','DS_MUN_SIT_BIOMETRIA','NR_ZONA','DS_ESTADO_CIVIL',
                         'DS_FAIXA_ETARIA','DS_GRAU_ESCOLARIDADE','QT_ELEITORES_PERFIL','QT_ELEITORES_BIOMETRIA','QT_ELEITORES_DEFICIENCIA')
    .distinct()
    .orderBy(F.col('NM_MUNICIPIO'), F.col('DS_GENERO'))
    .rdd.flatMap(lambda x: x)
    .collect()
)

# Mostrar os valores distintos
print(sorted(map(str, df_eleitorado_valores)))
'''

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
'''
Para realizar de todas as colunas é necessário poder computacional

# Verificar Valores Discrepantes das colunas df_resultado
df_resultado_final_valores =(
    df_result_final.select('NM_MUNICIPIO', 'ANO_ELEICAO', 'SG_UF','NR_TURNO','NR_ZONA','DS_ELEICAO',
                         'DS_CARGO_PERGUNTA','SG_PARTIDO','DS_TIPO_VOTAVEL','NM_VOTAVEL','QT_VOTOS', 'QT_ELEITORES_BIOMETRIA_NH')
    .distinct()
    .orderBy(col('NM_MUNICIPIO'), col('DS_GENERO'))
    .rdd.flatMap(lambda x: x)
    .collect()
)

# Mostrar os valores distintos
print(sorted(map(str, df_resultado_final_valores)))
'''

# Análise

In [ ]:
## Em qual município o cancidato X foi mais votado?
# Filtrar candidatos para DS_CARGO_PERGUNTA igual a "Prefeito"
prefeitos = df_result_final.filter(
    ((df_result_final.DS_CARGO_PERGUNTA == "Prefeito") |
    (df_result_final.DS_CARGO_PERGUNTA == "Vereador")) &
    (df_result_final.NM_VOTAVEL != "Nulo") |
    (df_result_final.NM_VOTAVEL != "Branco") &
    (df_result_final.SG_UF == "SP")
)

# Perguntar ao usuário o nome do candidato
candidato_input = input("Digite o nome do candidato: ")

# Filtrar votos do candidato informado pelo usuário
votos_candidato = prefeitos.filter(prefeitos.NM_VOTAVEL.like(f"%{candidato_input}%"))

# Encontrar o município onde o candidato foi mais votado
#mais_votado_municipio = votos_candidato.groupBy("NM_MUNICIPIO").agg(max("QT_VOTOS").alias("TOTAL_VOTOS")).orderBy("TOTAL_VOTOS", ascending=False).limit(1)
mais_votado_municipio = votos_candidato.groupBy("NM_VOTAVEL","DS_CARGO_PERGUNTA","NM_MUNICIPIO") \
    .agg(sum("QT_VOTOS").alias("TOTAL_VOTOS")) \
    .orderBy("TOTAL_VOTOS", ascending=False)

# Exibir o resultado do município mais votado
print(f"\nCandidato(s) que contém '{candidato_input}' no nome foi(ram) mais votado(s) no seguinte município:")
mais_votado_municipio.show()


Digite o nome do candidato: ANA

Candidato(s) que contém 'ANA' no nome foi(ram) mais votado(s) no seguinte município:
+--------------------+-----------------+---------------+-----------+
|          NM_VOTAVEL|DS_CARGO_PERGUNTA|   NM_MUNICIPIO|TOTAL_VOTOS|
+--------------------+-----------------+---------------+-----------+
|      IZAIAS SANTANA|         Prefeito|        JACAREÍ|      53978|
|         LUANA ALVES|         Vereador|      SÃO PAULO|      37550|
|        JANAÍNA LIMA|         Vereador|      SÃO PAULO|      30931|
|       MARIA GIOVANA|         Prefeito|      AMERICANA|      29562|
| ADRIANA DO HOSPITAL|         Prefeito|ITAQUAQUECETUBA|      29165|
|     JULIANA CARDOSO|         Vereador|      SÃO PAULO|      28402|
|     CORONEL ADRIANA|         Prefeito|     PIRACICABA|      22432|
|      SANDRA SANTANA|         Vereador|      SÃO PAULO|      19591|
|        JOÃO ANANIAS|         Vereador|      SÃO PAULO|      16821|
|      ADRIANA AFONSO|         Prefeito|      GUARULHO

'\n\nTestes\n\n# Encontrar o candidato mais votado em cada município\n#candidato_mais_votado_municipio = prefeitos.groupBy("NM_MUNICIPIO", "NM_VOTAVEL", "DS_CARGO_PERGUNTA").agg(max("QT_VOTOS").alias("TOTAL_VOTOS")).orderBy("NM_MUNICIPIO", ascending=True)\n\n# Exibir o resultado do candidato mais votado em cada município\n#print("\nCandidatos mais votados em cada município que contém o termo no nome:")\n#candidato_mais_votado_municipio.filter(col("NM_VOTAVEL").like(f"%{candidato_input}%")).show(truncate=False)'

In [ ]:
# Qual o candidato mais votado em cada municipio
(
    df_result_final
    .filter((
        df_result_final.NM_VOTAVEL != "Nulo")
        & (df_result_final.NM_VOTAVEL != "Branco")
        & (df_result_final.SG_UF == "SP")
    )
    .groupBy(
        'NM_VOTAVEL',
        'DS_CARGO_PERGUNTA',
        'SG_PARTIDO',
        'NM_MUNICIPIO'
    )
    .agg(
        F.sum('QT_VOTOS').alias("Total_votos")
    )
    .sort(
        'Total_votos',
        ascending=False
    )
    .withColumn(
        'rank',
        F.row_number().over(
            Window.partitionBy('DS_CARGO_PERGUNTA', 'NM_MUNICIPIO').orderBy(
                F.col('Total_votos').desc()
            )
        )
    )
    .filter(
        'rank = 1'
    )
    #Excluir a coulna rank
    .drop('rank')
    .sort(
        'Total_votos',
        ascending=False
    )
    .show(50, truncate=False)
)

+------------------------+-----------------+------------+---------------------+-----------+
|NM_VOTAVEL              |DS_CARGO_PERGUNTA|SG_PARTIDO  |NM_MUNICIPIO         |Total_votos|
+------------------------+-----------------+------------+---------------------+-----------+
|BRUNO COVAS             |Prefeito         |PSDB        |SÃO PAULO            |1754013    |
|PAULO SERRA             |Prefeito         |PSDB        |SANTO ANDRÉ          |266591     |
|ORLANDO MORANDO         |Prefeito         |PSDB        |SÃO BERNARDO DO CAMPO|261761     |
|GUTI                    |Prefeito         |PSD         |GUARULHOS            |261211     |
|FELICIO                 |Prefeito         |PSDB        |SÃO JOSÉ DOS CAMPOS  |204800     |
|ROGÉRIO LINS            |Prefeito         |PODE        |OSASCO               |204207     |
|PSDB                    |Vereador         |PSDB        |SÃO PAULO            |201577     |
|RUBENS FURLAN           |Prefeito         |PSDB        |BARUERI              |1

In [ ]:
# Teste Verificar quantidade de votos por Perfil
df_eleitorado_final.orderBy("NM_MUNICIPIO", "QT_ELEITORES_PERFIL").show(20)

+-----+---------------+--------------------+-------+---------+---------------+--------------------+--------------------+-------------------+----------------------+------------------------+
|SG_UF|   NM_MUNICIPIO|DS_MUN_SIT_BIOMETRIA|NR_ZONA|DS_GENERO|DS_ESTADO_CIVIL|     DS_FAIXA_ETARIA|DS_GRAU_ESCOLARIDADE|QT_ELEITORES_PERFIL|QT_ELEITORES_BIOMETRIA|QT_ELEITORES_DEFICIENCIA|
+-----+---------------+--------------------+-------+---------+---------------+--------------------+--------------------+-------------------+----------------------+------------------------+
|   GO|ABADIA DE GOIÁS|       Sem biometria|     56| FEMININO|          VIÚVO|40 a 44 anos     ...|ENSINO MÉDIO COMP...|                  1|                     1|                       0|
|   GO|ABADIA DE GOIÁS|       Sem biometria|     56| FEMININO|          VIÚVO|40 a 44 anos     ...|   SUPERIOR COMPLETO|                  1|                     1|                       0|
|   GO|ABADIA DE GOIÁS|       Sem biometria|     56| FE

In [ ]:
# Tests Agregação - Perfil Eleitorado mais votos em cada candidato
df_result_final_filtro = df_result_final.filter(
    (df_result_final.NM_VOTAVEL != "Nulo")
    & (df_result_final.NM_VOTAVEL != "Branco")
    & (df_result_final.NM_VOTAVEL == "CINTIA ALVES")
    & (df_result_final["NR_ZONA"] == 56)
).withColumn(
    "qt_total_votos",
    F.sum("QT_VOTOS").over(Window.partitionBy())
)
df_result_final_filtro.show(30)

+-----------+--------+--------------------+-----+-----------------+-------+-----------------+----------+---------------+------------+--------+-------------------------+--------------+
|ANO_ELEICAO|NR_TURNO|          DS_ELEICAO|SG_UF|     NM_MUNICIPIO|NR_ZONA|DS_CARGO_PERGUNTA|SG_PARTIDO|DS_TIPO_VOTAVEL|  NM_VOTAVEL|QT_VOTOS|QT_ELEITORES_BIOMETRIA_NH|qt_total_votos|
+-----------+--------+--------------------+-----+-----------------+-------+-----------------+----------+---------------+------------+--------+-------------------------+--------------+
|       2020|       1|Eleições Municipa...|   SP|BARÃO DE ANTONINA|     56|         Prefeito|        PL|        Nominal|CINTIA ALVES|      61|                        0|           392|
|       2020|       1|Eleições Municipa...|   SP|BARÃO DE ANTONINA|     56|         Prefeito|        PL|        Nominal|CINTIA ALVES|      53|                        0|           392|
|       2020|       1|Eleições Municipa...|   SP|BARÃO DE ANTONINA|     56|     

In [ ]:
# Tests Agregação
df_result_final_filtro = df_result_final.filter(
    (df_result_final.NM_VOTAVEL != "Nulo")
    & (df_result_final.NM_VOTAVEL != "Branco")
    & (df_result_final.NM_VOTAVEL == "CINTIA ALVES")
    & (df_result_final["NR_ZONA"] == 56)

)
# Join dos DataFrames
df_result_test = df_result_final_filtro.join(
    df_eleitorado_final,
    on=["NM_MUNICIPIO", "NR_ZONA"],
    how="inner"
).select(
     df_result_final["DS_CARGO_PERGUNTA"],
     df_result_final["NM_MUNICIPIO"],
     df_result_final["NM_VOTAVEL"],
     df_result_final["QT_VOTOS"],
     df_result_final["NR_ZONA"],
     df_eleitorado_final["QT_ELEITORES_PERFIL"],
     df_eleitorado_final["SG_UF"],
     df_eleitorado_final["DS_GENERO"],
     df_eleitorado_final["DS_ESTADO_CIVIL"],
     df_eleitorado_final["DS_FAIXA_ETARIA"],
     df_eleitorado_final["DS_GRAU_ESCOLARIDADE"]
)

df_result_test.show(30)

+-----------------+-----------------+------------+--------+-------+-------------------+-----+---------+---------------+--------------------+--------------------+
|DS_CARGO_PERGUNTA|     NM_MUNICIPIO|  NM_VOTAVEL|QT_VOTOS|NR_ZONA|QT_ELEITORES_PERFIL|SG_UF|DS_GENERO|DS_ESTADO_CIVIL|     DS_FAIXA_ETARIA|DS_GRAU_ESCOLARIDADE|
+-----------------+-----------------+------------+--------+-------+-------------------+-----+---------+---------------+--------------------+--------------------+
|         Prefeito|BARÃO DE ANTONINA|CINTIA ALVES|      61|     56|                  1|   SP|MASCULINO|       SOLTEIRO|16 anos          ...|ENSINO FUNDAMENTA...|
|         Prefeito|BARÃO DE ANTONINA|CINTIA ALVES|      61|     56|                  5|   SP|MASCULINO|       SOLTEIRO|16 anos          ...|ENSINO MÉDIO INCO...|
|         Prefeito|BARÃO DE ANTONINA|CINTIA ALVES|      61|     56|                 13|   SP|MASCULINO|       SOLTEIRO|17 anos          ...|ENSINO MÉDIO INCO...|
|         Prefeito|BARÃO DE 

In [ ]:
# Tests Agregação Resultado Eleição

df_zona_select = df_result_final.filter(
    (df_result_final["NR_ZONA"] == 56)
    & (df_result_final["SG_UF"] == "SP")

#& (df_eleitorado_final["NM_MUNICIPIO"] == "BARÃO DE ANTONINA")
)

result_perfil = df_zona_select.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(
    sum("QT_VOTOS").alias("SUM_QT_VOTOS")
)

# Mostrar Resultado
result_perfil.show(20)

#Deu diferença devido ao numero de apto vs o numero de quem comparaceu

+-----------------+-------+------------+
|     NM_MUNICIPIO|NR_ZONA|SUM_QT_VOTOS|
+-----------------+-------+------------+
|BARÃO DE ANTONINA|     56|        2101|
|         RIVERSUL|     56|        3470|
|       ITAPORANGA|     56|        8069|
+-----------------+-------+------------+



In [ ]:
# Tests Agregação Eleitorado
df_zona_select = df_eleitorado_final.filter(
    (df_eleitorado_final["NR_ZONA"] == 56)
    & (df_eleitorado_final["SG_UF"] == "SP")

#& (df_eleitorado_final["NM_MUNICIPIO"] == "BARÃO DE ANTONINA")
)

result_perfil = df_zona_select.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(
    sum("QT_ELEITORES_PERFIL").alias("SUM_QT_ELEITORES_PERFIL")
)

# Mostrar Resultado
result_perfil.show(20)


+-----------------+-------+-----------------------+
|     NM_MUNICIPIO|NR_ZONA|SUM_QT_ELEITORES_PERFIL|
+-----------------+-------+-----------------------+
|BARÃO DE ANTONINA|     56|                   2745|
|         RIVERSUL|     56|                   4645|
|       ITAPORANGA|     56|                  11476|
+-----------------+-------+-----------------------+

